# Functionality of this notebook: 

* calculate the ciCOH for normal and mild LFP data in rest

## Import Packages

In [1]:
import os, sys
import scipy.io as sio
import glob
import numpy as np
import matplotlib.pyplot as plt
import pickle 
import math
import pandas as pd
%matplotlib inline

In [2]:
# extract the exp folder path
currfolder = os.getcwd()
codefolder = currfolder[0 : currfolder.find('code')+len('code')]

# add path the exp folder
sys.path.append(codefolder)

# import_nbmodule used for import package in .ipynb
import import_nbmodule

# import util/folder_extract.pynb 
from util.folder_extract import exp_subfolders, code_corresfolder

# import ciCoherence_overtime in connAnalyTool/synchronization_indices.ipynb
from connAnalyTool.synchronization_indices import ciCoherence_acrosstrials
from connAnalyTool.synchronization_indices import ciCoherence_overtime

## exp subfolders & code_corresfolder

In [3]:
_, _, pipelinefolder, _= exp_subfolders()

In [4]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [5]:
nb_name = nb_name[0: nb_name.find('.ipynb')]

# corresfolder
corresfolder, correparentfolder = code_corresfolder(os.getcwd(), nb_name)

## global parameters

In [6]:
freq = [26, 28]

animal = 'Pinky'

phase = 'base'

fs = 500

## input setup

In [7]:
inputfolder = os.path.join(pipelinefolder, 'NHPs', animal, '0_dataPrep', 'SKT', 
                           'm2_STKData_narrowfiltered' + str(freq[0]) + '_' + str(freq[1]))

In [8]:
variablesinLoadfile = ['lfpdata', 'fs', 'chnAreas']

In [9]:
chnInf_folder = os.path.join(pipelinefolder, 'NHPs', animal, '1_dataAnaly', 'FCAnaly','SKT')
chnInf_file = os.path.join(chnInf_folder, 'chn_brainArea_simCoord_BrainArea.csv')

## Save Setup

In [10]:
savefolder = corresfolder
savefilename =  'ciCOH_SKT' + '_freq' + str(freq[0]) + '_' + str(freq[1])

## extract lfp

In [11]:
def lfpallfiles_extract(files):
    if 'lfpdatas' in locals():
        del lfpdata
    
    for i, file in enumerate(files):
        
        ### load data
        matdat = sio.loadmat(file, variable_names = variablesinLoadfile, 
                             struct_as_record = False, squeeze_me = True) 
        
        
        ### extract the noused channels, only calculate once
        if i == 0:
            
            # chnAreas
            chnAreas = matdat['chnAreas'].tolist()
            
            # fs: sample rate
            fs = matdat['fs'] 
             
        

        ### dealing lfp data
        
        # lfp (np.ndarray): ntemporal * nchns * ntrials
        lfpdata_1file = matdat['lfpdata']
        
        # concatenate to lfpdata for all files
        if 'lfpdatas' not in locals():
            lfpdatas = lfpdata_1file
        else:
            lfpdatas = np.concatenate((lfpdatas, lfpdata_1file), axis = 2)
          
    
    return lfpdatas, chnAreas

In [12]:
files_normal = glob.glob(os.path.join(inputfolder, '*_normal_*'))
files_mild = glob.glob(os.path.join(inputfolder, '*_mild_*'))
files_moderate = glob.glob(os.path.join(inputfolder, '*_moderate_*'))

print(len(files_normal))
print(len(files_mild))
print(len(files_moderate))

0
9
0


In [ ]:
lfptrials_normal, chnAreas = lfpallfiles_extract(files_normal)
lfptrials_mild, _ = lfpallfiles_extract(files_mild)
lfptrials_moderate, _ = lfpallfiles_extract(files_moderate)

In [ ]:
if phase == 'move':
    lfptrials_normal = lfptrials_normal[500:, :, :]
    lfptrials_mild = lfptrials_mild[500:, :, :]
    lfptrials_moderate = lfptrials_moderate[500:, :, :]
    
if phase == 'base':
    lfptrials_normal = lfptrials_normal[0:500, :, :]
    lfptrials_mild = lfptrials_mild[0:500, :, :]
    lfptrials_moderate = lfptrials_moderate[0:500:, :, :]
    
    
print(lfptrials_normal.shape)
print(lfptrials_mild.shape)
print(lfptrials_moderate.shape)

print(chnAreas)

## Assign the xy coord of each chnArea

In [ ]:
# load channel coord from chnInf_file
df = pd.read_csv(chnInf_file, header = 0)

# fill in the x,y coordinates of each area in chnAreas based on the values in df_chninf
coord_x, coord_y = np.zeros(shape = [len(chnAreas), ]), np.zeros(shape = [len(chnAreas), ])
for i, chnArea in enumerate(chnAreas):
    
    mask_area = (df['brainarea']==chnArea)

    x, y = df['simulated_x'][mask_area].to_numpy(), df['simulated_y'][mask_area].to_numpy()

    coord_x[i], coord_y[i] = x, y
    
    del mask_area, x, y

df_chninf = pd.DataFrame(data = {'chnAreas': chnAreas, 'coord_x': coord_x, 'coord_y': coord_y})
    
del df, coord_x, coord_y

df_chninf

## Calculate ciCOH

### balance mild, normal and moderate trials

In [ ]:
# # select the smaller trial number
# ntrials_normal, ntrials_mild = lfpwins_normal.shape[2], lfpwins_mild.shape[2]
# ntrials_moderate = lfpwins_moderate.shape[2]

# ntrials = min([ntrials_normal, ntrials_mild, ntrials_moderate])

# # balance trials by randomly selecting ntrials
# idx_ntrials = np.random.randint(ntrials_normal, size = ntrials)
# lfpwins_normal = lfpwins_normal[:,:,idx_ntrials]

# idx_ntrials = np.random.randint(ntrials_mild, size = ntrials)
# lfpwins_mild = lfpwins_mild[:,:,idx_ntrials]

# idx_ntrials = np.random.randint(ntrials_moderate, size = ntrials)
# lfpwins_moderate = lfpwins_moderate[:,:,idx_ntrials]

###  normal ciCOH

In [ ]:
lfptrials_allfiles = lfptrials_normal

### calculate ciCOH
ntemp, nchns, ntrials = lfptrials_allfiles.shape

ciCOH = np.zeros((nchns, nchns))
for chni in range(nchns -1):
    
    sig1 = lfptrials_allfiles[:, chni, :] # sig1: ntemp * ntrials
    sig1 = np.transpose(sig1) # sig1: ntrials * ntemp
    
    for chnj in range(chni+1, nchns):
        
        sig2 = lfptrials_allfiles[:, chnj, :]
        sig2 = np.transpose(sig2) # sig2: ntrials * ntemp
        
        ciCOHs = ciCoherence_acrosstrials(sig1, sig2) # ciCOHs: 1 * ntemp
        
        # average across time
        ciCOH[chni, chnj] = np.mean(ciCOHs)
        
        # symmetrical
        ciCOH[chnj, chni] = ciCOH[chni, chnj]
        
        del sig2
    del sig1
    
ciCOH_normal = ciCOH

del ciCOH, lfptrials_allfiles

### mild ciCOH

In [ ]:
lfptrials_allfiles = lfptrials_mild

### calculate ciCOH
ntemp, nchns, ntrials = lfptrials_allfiles.shape

ciCOH = np.zeros((nchns, nchns))
for chni in range(nchns -1):
    
    sig1 = lfptrials_allfiles[:, chni, :] # sig1: ntemp * ntrials
    sig1 = np.transpose(sig1) # sig1: ntrials * ntemp
    
    for chnj in range(chni+1, nchns):
        
        sig2 = lfptrials_allfiles[:, chnj, :]
        sig2 = np.transpose(sig2) # sig2: ntrials * ntemp
        
        ciCOHs = ciCoherence_acrosstrials(sig1, sig2) # ciCOHs: 1 * ntemp
        
        # average across time
        ciCOH[chni, chnj] = np.mean(ciCOHs)
        
        # symmetrical
        ciCOH[chnj, chni] = ciCOH[chni, chnj]
        
        del sig2
    del sig1
    
ciCOH_mild = ciCOH

del ciCOH, lfptrials_allfiles

### moderate ciCOH

In [ ]:
lfptrials_allfiles = lfptrials_moderate

### calculate ciCOH
ntemp, nchns, ntrials = lfptrials_allfiles.shape

ciCOH = np.zeros((nchns, nchns))
for chni in range(nchns -1):
    
    sig1 = lfptrials_allfiles[:, chni, :] # sig1: ntemp * ntrials
    sig1 = np.transpose(sig1) # sig1: ntrials * ntemp
    
    for chnj in range(chni+1, nchns):
        
        sig2 = lfptrials_allfiles[:, chnj, :]
        sig2 = np.transpose(sig2) # sig2: ntrials * ntemp
        
        ciCOHs = ciCoherence_acrosstrials(sig1, sig2) # ciCOHs: 1 * ntemp
        
        # average across time
        ciCOH[chni, chnj] = np.mean(ciCOHs)
        
        # symmetrical
        ciCOH[chnj, chni] = ciCOH[chni, chnj]
        
        del sig2
    del sig1
    
ciCOH_moderate = ciCOH

del ciCOH, lfptrials_allfiles

## save ciCOH

In [ ]:
ciCOH = dict()
ciCOH['normal'], ciCOH['mild']  = ciCOH_normal, ciCOH_mild
ciCOH['moderate']  = ciCOH_moderate


setup = dict()
setup['fs'] = fs

setup['ntemp_normal'] = lfptrials_normal.shape[0]
setup['ntemp_mild'] = lfptrials_mild.shape[0]
setup['ntemp_moderate'] = lfptrials_moderate.shape[0]
setup['ntrials_normal'] = lfptrials_normal.shape[2]
setup['ntrials_mild'] = lfptrials_mild.shape[2]
setup['ntrials_moderate'] = lfptrials_moderate.shape[2]

fc = dict()
fc['ciCOH'] = ciCOH
fc['chnInf'] = df_chninf
fc['setup']= setup

In [ ]:
with open(os.path.join(savefolder, savefilename + '.pickle'), 'wb') as fp:
    pickle.dump(fc, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
os.path.join(savefolder, savefilename)